In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time

from helper import *

In [ ]:
#Loading merged dataset

title_basics = pd.read_csv('../data/title_basics_data.tsv', sep = '\t', low_memory=False)
title_basics.replace('\\N', np.nan, inplace=True)
title_basics = title_basics.dropna(subset=['startYear'])
title_basics['startYear'] = title_basics['startYear'].astype('int')
title_basics = title_basics[(title_basics['titleType']=='movie') & (title_basics['startYear']>2013)]

print(title_basics.shape, title_basics['tconst'].nunique())

proxy_list = pd.read_csv('./Free_Proxy_List.csv')

proxy_list['full']=proxy_list['ip'].astype('string')+':'+proxy_list['port'].astype('string')
proxies = proxy_list['full'].tolist()

In [ ]:
# Check if the progress file exists
if os.path.exists("progress_ratings.txt"):
    with open("progress_ratings.txt", "r") as file:
        start_index = int(file.read())
else:
    start_index = 0

# Check if the ratings data file exists
if os.path.exists("df_ratings.csv"):
    df_ratings = pd.read_csv("df_ratings.csv")
    ratings = df_ratings["ratings"].tolist()
    films = df_ratings["tconst"].tolist()
else:
    ratings = []
    films = []

# Scraping data from boxofficemojo.com
new_films = title_basics['tconst'].unique()[start_index:]

break_counter = 0

for i, film in enumerate(new_films, start=start_index):
    print("=" * 80)
    if break_counter == 10:
        print("More than 10 bad requests")
        break

    proxy = random.choice(proxies)
    print(f"Scraping for {film}")
    rating = get_user_ratings(film, proxy)
    ratings.append(rating)
    films.append(film)
    time.sleep(random.uniform(0, 0.8))

    # Save data after processing 5 films
    if i % 5 == 0:
        with open("progress_ratings.txt", "w") as file:
            file.write(str(i + 1))
        df_ratings = pd.DataFrame({"tconst": films, "ratings": ratings})
        df_ratings.to_csv("df_ratings.csv", index=False)
        print("df_ratings updated")
        time.sleep(random.uniform(1,3))

# Remove the progress file as the processing is complete
os.remove("progress_ratings.txt")

In [ ]:
# tconst = 'tt0000003'
# proxy = proxies[3]



In [ ]:
# url = f"https://www.imdb.com/title/{tconst}/reviews"

# # List of user agent headers
# user_agents = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
#     "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59"
# ]


# rating_and_username = []


# # Get the current user agent
# current_user_agent = user_agents[1]

# # Set the headers for the request
# headers = {"User-Agent": current_user_agent}

# # Create a session object
# session = requests.Session()

# # Set the proxy for the session
# session.proxies = {'http': proxy}

# # Make the request using rotating proxies
# res = session.get(url, headers=headers, timeout=5)

# if res.status_code==200:
#     print(f"Proxy {proxy} successfully connected")
#     soup = BeautifulSoup(res.content, 'html.parser')


#     chunks = re.findall(pattern = r'<div class="review-container">(.*?)Was this review helpful\?', string=str(soup), flags=re.DOTALL)

#     for chunk in chunks:
#         if "Warning: Spoilers" in chunk:
#             continue
#         else:
#             rating_match = re.search(r'<span>(\d{1})<\/span><span class="point-scale">', chunk)
#             rating = rating_match.group(1) if rating_match else None
#             if rating_match == None:
#                 continue

#             username_match = re.search(r'<span class="display-name-link"><a href="\/user\/(.*?)\?ref_=tt_urv', chunk)
#             username = username_match.group(1).strip("/") if username_match else None
            
#             rating_and_username.append((username, rating))
#     print(f"Successfully scraped: {rating_and_username}")

# else:
#     print(print(f"Request for {tconst} failed to execute."))






In [ ]:
# tconst = "tt9362722"
# url = f"https://www.imdb.com/title/{tconst}/reviews"


# proxy = "103.127.29.105:7497"

# user_agents = [
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
#         # Add more user agents here...
#     ]

# # Get the current user agent
# current_user_agent = user_agents[0]

# # Set the headers for the request
# headers = {"User-Agent": current_user_agent}

# session = requests.Session()

# # Set the proxy for the session
# session.proxies = {'http': proxy}

# res = session.get(url, headers=headers, timeout=5)


# soup = BeautifulSoup(res.content, 'html.parser')


1. Scrape users only who commented on all reviews
2. Loop through each user and scrape <br>
<font color = 'red'> Explore double-layered proxies